<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/1_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting UVI with LSTMs

[Notebook 0: Data Cleaning](./0_DataCleaning.ipynb)

[Notebook 1: EDA](./1_EDA.ipynb)

[Notebook 2: Modeling and Predictions](./2_Modeling_and_Predictions.ipynb)

[Notebook 3: Technical Report](./3_Technical_Report.ipynb)

Was wird in diesem Notebook gemacht?

## Verbinden mit der Google Drive und Import der benötigten Module:

In [24]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
## import modules

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
from datetime import timedelta

import matplotlib
import seaborn as sns

In [26]:
latitude = 50.2
longitude = 7.8

seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

In [49]:
# Pfad zur CSV-Datei auf Google Drive
name_Messwerte = 'Messdaten.csv'
name_Vorhersage = 'Vorhersage.csv'
drive_path = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'
output_plot_path = '/content/drive/My Drive/plot_daily_UVI'

In [30]:
# Import der Daten für die EDA
df_UVI = pd.read_csv(drive_path + name_Messwerte)
df_CAMS = pd.read_csv(drive_path + name_Vorhersage)

# Messzeitpunkt in Datetime umwandeln
df_UVI['Datetime'] = pd.to_datetime(df_UVI['Datetime'])
df_CAMS['Datetime'] = pd.to_datetime(df_CAMS['Datetime'])

df_UVI.set_index('Datetime', inplace=True)
df_CAMS.set_index('Datetime', inplace=True)

In [38]:
df_UVI_hourly = df_UVI[['UVI','erythem', 'SolarZenithAngle']].resample('H').mean()

## Plotten der Dataframes:

### Plot der Tagesdurchschnitte:

In [40]:
def plot_daily_mean_data(df1, df2, value_column1='UVI', value_column2='Wert2'):

    # Resample auf tägliche Auflösung und Berechnung des Durchschnitts
    df1_daily = df1.resample('D').mean()
    df2_daily = df2.resample('D').mean()

    # Erstellen Sie den Plot
    plt.figure(figsize=(12, 6))
    plt.plot(df1_daily.index, df1_daily[value_column1], label='DataFrame 1', marker='o')
    plt.plot(df2_daily.index, df2_daily[value_column2], label='DataFrame 2', marker='x')

    plt.title('Tägliche Messwerte')
    plt.xlabel('Datum')
    plt.ylabel('Durchschnittlicher Messwert')
    plt.legend()
    plt.grid(True)

    # Rotieren Sie die X-Achsenbeschriftungen für bessere Lesbarkeit
    plt.xticks(rotation=45)

    # Zeigen Sie den Plot an
    plt.tight_layout()
    plt.show()

In [ ]:
plot_daily_mean_data(df_UVI, df_CAMS)

### Plot der täglichen Verläufe:

In [47]:
def plot_daily_data_and_save(df1, df2, output_dir, columns1=['Wert1'], columns2=['Wert2']):

    # Erstellen Sie das Ausgabeverzeichnis, wenn es nicht vorhanden ist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Gruppieren Sie die DataFrames nach Tag und plotten Sie den Verlauf für jeden Tag
    for date, group1 in df1.groupby(df1.index.date):
        group2 = df2[df2.index.date == date]

        # Erstellen Sie Subplots für jeden Wert
        fig, axes = plt.subplots(len(columns1) + len(columns2), 1, figsize=(10, 6 * (len(columns1) + len(columns2))))

        for i, col in enumerate(columns1):
            axes[i].plot(group1.index, group1[col], label=f'DataFrame 1 - {col}')
            axes[i].set_ylabel(f'{col} (DataFrame 1)')
            axes[i].grid(True)

        for i, col in enumerate(columns2, start=len(columns1)):
            axes[i].plot(group2.index, group2[col], label=f'DataFrame 2 - {col}')
            axes[i].set_ylabel(f'{col} (DataFrame 2)')
            axes[i].grid(True)

        plt.xlabel('Stunde')
        plt.suptitle(f'Messwerte für {date}', y=0.92)  # Einstellen des Titels oberhalb der Subplots
        plt.tight_layout()

        # Speichern Sie den Plot als Bild
        plot_filename = os.path.join(output_dir, f'plot_{date}.png')
        plt.savefig(plot_filename)
        plt.close()  # Schließen Sie den aktuellen Plot, um Speicher freizugeben

In [48]:
columns_UVI = ['UVI', 'erythem']  # Beispiel für Spalten aus df1
columns_CAMS = ['aod469', 'aod550']  # Beispiel für Spalten aus df2
plot_daily_data_and_save(df_UVI, df_CAMS, output_directory, columns1, columns2)

NameError: ignored